In [ ]:
%%html

<style>
.jp-MarkdownOutput {
    font-size: 2.5em !important;
}
.jp-MarkdownOutput table {
    font-size: 0.75em !important;
}
.jp-OutputArea-output pre {
    font-size: 2em !important;
}
.cm-content {
    font-size: 2em !important;
}
.page-id-xx, html {
    scrollbar-width: none; /* FF */
}
::-webkit-scrollbar {
    width: 0px; /* Chrome & Edge */
}
.jp-Notebook.jp-mod-commandMode .jp-Cell.jp-mod-selected {
    background: none;
}
</style>